In [1]:
import pandas as pd
from wrappers import *
import os
from bs4 import BeautifulSoup
import zipfile
import shutil
import requests

In [6]:
"""
Demo for central library scraping
"""
from updated_scraper import *
s = PkuScraper()

# Important:
# After you log in with Chrome, with VPN on and pkulaw signed in, right click anywhere on the page and direct to "Inspect", find "Network", zoom into one event, and paste the Cookie here
s.set_input_type('keyword')
s.set_cookies('pkulaw_v6_sessionid=qm4iqggkoktau3szdek4bfie; Hm_lvt_8266968662c086f34b2a3e2ae9014bf8=1630066629,1631060463; TKMessage=YAfPC0z71+fcKpSCFfGGegieH8itqHxV2OqOO6H79fl5/3AtqBO+iMEeQW8s06LxYZrtBmm5VPUeQVUkP/cHP3K1fLrZVN28PwpK9nfCGVQWxLyluZIQ2cWU4gCBVuQN2+C35U7SA8m9jM4weX14GkFsigG+tYTAnp/4QLA8MbflunEn8Hr3eE8PLh1DYnFWtFRNc9aGVOZ3ARaHHBZ1CL1A98OVGwY20YV4fqQBwrqPAvEkEZaov1yqxDarlkjR; authormes=5f8e703755203640a7baa1c56e09ce5616c7268dca732f3b39028526dff6560a10f8eedcbd05b937bdfb; Hm_lpvt_8266968662c086f34b2a3e2ae9014bf8=1632072563; xCloseNew=20')
s.library = 'lar' # lar: library for local govt documents. chl: library for central documents
s.set_filters(lib_='lar', type_=['unspecified'], dept_=['unspecified'], effective_=['unspecified']) # For other choices please refer to the code


In [7]:
s.edit_postform({'Aggs.Category': '001'})
soup = s.search_page_V6('', _debug=True)

In [9]:
soup

<!-- 面包屑 --><html><body><div class="crumbs">
<div id="navbar">
<!--首页-->
<a href="https://www.pkulaw.com" logcode="005001">首页</a>
<!--栏目-->
<i>&gt;</i>
<a href="/law" logcode="005001">法律法规</a>
<i>&gt;</i>
<a href="/law/lar" logcode="005001">地方法规</a>
<a class="explain" href="https://www.pkulaw.com/helps/23.html" logclick="" logcode="012001" target="_blank"><i class="c-icon c-tips"></i>本库说明</a>
</div>
</div>
<!-- 检索条件 -->
<div class="search-condition-wrap">
<div>
<div class="box">
<div class="condition">
<span>检索条件：</span>
<!--跳转-->
<!--分组统计-->
<a class="crumb-select-item" cluster_code="001" cluster_index="5" href="javascript:;" logcode="011001">
<i class="c-icon close"></i>
<b>法规类别：</b>
                    宪法
                </a>
<!--筛选项-->
<!--分组项更多-->
<input id="isShowResultSearch" type="hidden" value="true"/>
</div>
<div class="oparetion">
<a href="javascript:void(0);" id="btn-clear"><i class="c-icon c-clear"></i>清空</a>
<a href="javascript:void(0);" id="btn-save"><i class="c-icon c-s

In [ ]:

# One first thing we can do is to simulate the searching process:
print(s.search_page_V6('证照分离'))
# It will return a list where one of the sublists document the titles, one document the urls, and the other the id's

# Once you get a url, we can get the full text of the paper from this function call
print(s.decode_page_v6(r"https://www.pkulaw.com/chl/7e71a76b28fb96efbdfb.html?keyword=%E8%AF%95%E7%82%B9"))
# It returns a Soup object containing the texts of documents

# If we don't want to visit the website repeatedly for those documents, we can download them once and for all.
s.set_ouput_directory(r'/Users/kevin/Desktop/temp.zip')
s.download('f3c7b1f7db0795fcbdfb,69425ff77e3fd798bdfb,a8decc3fc18fb591bdfb,2c8dfe7452186879bdfb,78c556b1def5fb57bdfb,87899185ab4ffaadbdfb')

In [8]:
# You might also want to unzip the documents you downloaded.
def un_zip(file_name):
    # unzip zip file
    if not file_name.endswith('.zip'):
        return
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        try:
            zip_file.extract(names, file_name + r"_files/")
        except:
            pass
    zip_file.close()

In [30]:

raise ValueError
# ===========================================================
# DO NOT RUN: Some sample code for reference (how to unzip)
# Init Dirs
src_path = '/Users/kevin/Desktop/tmp.zip_files/'
dest_path = '/Users/kevin/Dropbox/Policy Experimentation/Data/Raw Webpages/Local/'
zip_path = '/Users/kevin/Desktop/tmp.zip'

counter = 0
maxlen = df_local.shape[0]
while counter < maxlen:
    download(','.join(df_local['gid'][counter:counter+10]))
    un_zip(zip_path)
    file_list = os.listdir(src_path)
    if len(file_list) > 0:
        for file in file_list:
            counter += 1
            #time.sleep(1)
            if counter % 100 == 0:
                print('Progress:', counter)
                time.sleep(20)
            try:
                # unzip and move directory
                os.rename(src_path+file, src_path + str(counter) + '.html')
                shutil.move(src_path + str(counter) + '.html', dest_path + str(counter) + '.html')
            except:
                pass
# ===========================================================    

BadZipFile: File is not a zip file